In [16]:
import tflscripts
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import numpy as np
import pandas as pd

tflscripts.set_dataset_folder('../../datasets/')

configuration = tflscripts.read_configuration()

activities = configuration['analysed_activities']
activities_i = [configuration['activities'].index(a) for a in activities]

In [23]:
def read_dataset(device, dataset, scale=True):
    df, df_labels = tflscripts.read_and_filter_dataset(
            dataset + '-1s',
            device,
            use_features='.*',
            use_activities=activities_i,
            check_all_activities=False,
            scale=scale,
            with_feature_selection=False)

    df = df.loc[df.index.isin(df_labels.index)]
    df_labels = df_labels.loc[df_labels.index.isin(df.index)]

    return df, df_labels

def test(source_dataset, source_device, target_dataset, target_device, label, features, scale_independently):

    df_source, df_source_labels = read_dataset(dataset=source_dataset,
                                               device=source_device,
                                               scale=scale_independently)
    df_target, df_target_labels = read_dataset(dataset=target_dataset,
                                               device=target_device,
                                               scale=scale_independently)

    c1 = df_source.filter(regex=features).columns.tolist()
    c2 = df_target.filter(regex=features).columns.tolist()
    columns = [i for i in c1 if i in c2]

    x_train = df_source[columns]
    y_train = tflscripts.get_y_for_label(df_source_labels, label)

    x_test = df_target[x_train.columns]
    y_test = tflscripts.get_y_for_label(df_target_labels, label)
    
    clf = svm.SVC(kernel='linear', decision_function_shape='ovr')

    if not scale_independently:
        scaler = StandardScaler()
        concated = pd.concat([x_train, x_test])
        scaler.fit(concated)
        x_train[x_train.columns] = scaler.transform(x_train[x_train.columns])
        x_test[x_test.columns] = scaler.transform(x_test[x_test.columns])

    ppl = Pipeline([
        ('impute', Imputer()),
        ('clf', clf)
    ])
    ppl.fit(x_train, y_train)

    predicted = ppl.predict(x_test)

    mask_negative = y_test == -1
    accuracy_negative = accuracy_score(y_test[mask_negative], predicted[mask_negative])
    
    mask_positive = y_test != -1
    accuracy_positive = accuracy_score(y_test[mask_positive], predicted[mask_positive])
    
    accuracy = accuracy_score(y_test, predicted)
    
    print('Accuracy', accuracy)
    print('Accuracy positive', accuracy_positive)
    print('Accuracy negative', accuracy_negative)

In [20]:
source_dataset = 'synergy-final-iter4'
source_device = '128.237.227.76'
target_dataset = 'synergy-final-iter1'
target_device = '128.237.254.195'

label_name = 'Dishes'
label = configuration['activities'].index(label_name)

print('Independent scaling')

test(source_dataset,
     source_device,
     target_dataset,
     target_device,
     label=label,
     features='.*',
     scale_independently=True)

print()
print('Common scaling')

test(source_dataset,
     source_device,
     target_dataset,
     target_device,
     label=label,
     features='.*',
     scale_independently=False)

Independent scaling
Accuracy 0.932528113286
Accuracy positive 0.58961038961
Accuracy negative 0.998015873016

Common scaling
Accuracy 0.875468554769
Accuracy positive 0.223376623377
Accuracy negative 1.0


In [24]:
source_dataset = 'synergy-final-iter2'
source_device = '128.237.248.186'
target_dataset = 'synergy-final-iter1'
target_device = '128.237.254.195'

label_name = 'Chopping food'
label = configuration['activities'].index(label_name)

print('Independent scaling')

test(source_dataset,
     source_device,
     target_dataset,
     target_device,
     label=label,
     features='.*',
     scale_independently=True)

print()
print('Common scaling')

test(source_dataset,
     source_device,
     target_dataset,
     target_device,
     label=label,
     features='.*',
     scale_independently=False)

Independent scaling
Accuracy 0.900874635569
Accuracy positive 0.539325842697
Accuracy negative 0.963814180929

Common scaling
Accuracy 0.619325281133
Accuracy positive 0.86797752809
Accuracy negative 0.576039119804


In [18]:
source_dataset = 'scott-final-iter1'
source_device = '128.237.248.186'
target_dataset = 'synergy-final-iter1'
target_device = '128.237.254.195'

label_name = 'Dishes'
label = configuration['activities'].index(label_name)

print('Independent scaling')

test(source_dataset,
     source_device,
     target_dataset,
     target_device,
     label=label,
     features='.*',
     scale_independently=True)

print()
print('Common scaling')

test(source_dataset,
     source_device,
     target_dataset,
     target_device,
     label=label,
     features='.*',
     scale_independently=False)

Independent scaling
Accuracy 0.989171178676
Accuracy positive 0.932467532468
Accuracy negative 1.0

Common scaling
Accuracy 0.995835068721
Accuracy positive 0.979220779221
Accuracy negative 0.999007936508


In [19]:
source_dataset = 'scott-final-iter1'
source_device = '128.237.248.186'
target_dataset = 'scott-final-iter1'
target_device = 'Matrix b827eb41f96f'

label_name = 'Dishes'
label = configuration['activities'].index(label_name)

print('Independent scaling')

test(source_dataset,
     source_device,
     target_dataset,
     target_device,
     label=label,
     features='.*',
     scale_independently=True)

print()
print('Common scaling')

test(source_dataset,
     source_device,
     target_dataset,
     target_device,
     label=label,
     features='.*',
     scale_independently=False)

Independent scaling
Accuracy 0.875253961804
Accuracy positive 0.298630136986
Accuracy negative 0.975667938931

Common scaling


/home/giotto/.local/lib/python3.5/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/giotto/.local/lib/python3.5/site-packages/pandas/core/frame.py:2454: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc._setitem_with_indexer((slice(None), indexer), value)
/home/giotto/.local/lib/python3.5/site-packages/pandas/core/frame.py:2424: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setit

Accuracy 0.14831369362
Accuracy positive 1.0
Accuracy negative 0.0


In [10]:
source_dataset = 'scott-final-iter1'
source_device = '128.237.248.186'
target_dataset = 'scott-final-iter1'
target_device = 'xdk_2'

label_name = 'Dishes'
label = configuration['activities'].index(label_name)

print('Independent scaling')

test(source_dataset,
     source_device,
     target_dataset,
     target_device,
     label=label,
     features='.*',
     scale_independently=True)

print()
print('Common scaling')

test(source_dataset,
     source_device,
     target_dataset,
     target_device,
     label=label,
     features='.*',
     scale_independently=False)

Independent scaling
Accuracy 0.948624595469
Accuracy positive 1.0
Accuracy negative 0.939667458432

Common scaling


/home/giotto/.local/lib/python3.5/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/giotto/.local/lib/python3.5/site-packages/pandas/core/frame.py:2454: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc._setitem_with_indexer((slice(None), indexer), value)
/home/giotto/.local/lib/python3.5/site-packages/pandas/core/frame.py:2424: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setit

Accuracy 0.148462783172
Accuracy positive 1.0
Accuracy negative 0.0


In [23]:
source_dataset = 'scott-final-iter1'
source_device = '128.237.248.186'
target_dataset = 'synergy-final-iter2'
target_device = 'xdk_1'

label_name = 'Dishes'
label = configuration['activities'].index(label_name)

print('Independent scaling')

test(source_dataset,
     source_device,
     target_dataset,
     target_device,
     label=label,
     features='.*',
     scale_independently=True)

print()
print('Common scaling')

test(source_dataset,
     source_device,
     target_dataset,
     target_device,
     label=label,
     features='.*',
     scale_independently=False)

Independent scaling
Accuracy 0.881549815498
Accuracy positive 1.0
Accuracy negative 0.862349914237

Common scaling
Accuracy 0.326568265683
Accuracy positive 0.746031746032
Accuracy negative 0.258576329331


In [24]:
source_dataset = 'scott-final-iter1'
source_device = 'xdk_2'
target_dataset = 'synergy-final-iter2'
target_device = 'xdk_1'

label_name = 'Dishes'
label = configuration['activities'].index(label_name)

print('Independent scaling')

test(source_dataset,
     source_device,
     target_dataset,
     target_device,
     label=label,
     features='.*',
     scale_independently=True)

print()
print('Common scaling')

test(source_dataset,
     source_device,
     target_dataset,
     target_device,
     label=label,
     features='.*',
     scale_independently=False)

Independent scaling
Accuracy 0.930258302583
Accuracy positive 0.997354497354
Accuracy negative 0.919382504288

Common scaling
Accuracy 0.909594095941
Accuracy positive 0.465608465608
Accuracy negative 0.981560891938


In [25]:
source_dataset = 'scott-final-iter1'
source_device = 'Matrix b827eb41f96f'
target_dataset = 'synergy-final-iter2'
target_device = 'Matrix b827ebe6e0f8'

label_name = 'Dishes'
label = configuration['activities'].index(label_name)

print('Independent scaling')

test(source_dataset,
     source_device,
     target_dataset,
     target_device,
     label=label,
     features='.*',
     scale_independently=True)

print()
print('Common scaling')

test(source_dataset,
     source_device,
     target_dataset,
     target_device,
     label=label,
     features='.*',
     scale_independently=False)

Independent scaling
Accuracy 0.870062753784
Accuracy positive 0.52380952381
Accuracy negative 0.926211926212

Common scaling
Accuracy 0.860465116279
Accuracy positive 0.0
Accuracy negative 1.0
